In [1]:
import os
import sys
import json
import requests
import pandas as pd
from io import StringIO
from tqdm import tqdm_notebook as tqdm_nb

In [2]:
exomol_all_url = 'http://www.exomol.com/db/exomol.all'

content = requests.get(exomol_all_url).text.replace('#','')
exomol_col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6']
exomol_all = pd.read_csv(StringIO(content), sep='\\s+', names=exomol_col_name, header=None)

exomol_all

,c0,c1,c2,c3,c4,c5,c6
0,EXOMOL.master,ID,NaN,NaN,NaN,NaN,NaN
1,20200720,Version,number,with,format,YYYYMMDD,NaN
2,129,Number,of,molecules,in,the,database
3,11,Number,of,molecule,names,listed,NaN
4,Water,Name,of,the,molecule,NaN,NaN
...,...,...,...,...,...,...,...
1309,XLYOFNOQVPJJNP-UHFFFAOYSA-O,Inchi,key,of,isotopologue,NaN,NaN
1310,1H3-16O_p,Iso-slug,NaN,NaN,NaN,NaN,NaN
1311,(1H)3(16O)+,IsoFormula,NaN,NaN,NaN,NaN,NaN
1312,eXeL,Isotopologue,dataset,name,NaN,NaN,NaN


In [3]:
first = exomol_all['c1']
third = exomol_all['c3']
row = len(first)


iso_slug = pd.DataFrame()
iso_formula = pd.DataFrame()
isotopologue = pd.DataFrame()
molecule_single = pd.DataFrame()
num_isotopologues = pd.DataFrame()


for i in tqdm_nb(range(row)):
        
    _iso_slug = exomol_all[first.isin(['Iso-slug'])]['c0'].values
    _iso_formula = exomol_all[first.isin(['IsoFormula'])]['c0'].values
    _isotopologue = exomol_all[first.isin(['Isotopologue'])]['c0'].values
    _molecule_single = exomol_all[first.isin(['Molecule'])]['c0'].values
    _num_isotopologues = exomol_all[third.isin(['isotopologues'])]['c0'].values


iso_slug = iso_slug.append(pd.DataFrame(_iso_slug))
iso_formula = iso_formula.append(pd.DataFrame(_iso_formula))
isotopologue = isotopologue.append(pd.DataFrame(_isotopologue))
molecule_single = molecule_single.append(pd.DataFrame(_molecule_single))
num_isotopologues = num_isotopologues.append(pd.DataFrame(_num_isotopologues))


In [4]:
iso_slug

,0
0,2H2-16O
1,1H2-16O
2,1H2-18O
3,1H2-17O
4,1H-2H-16O
...,...
183,28Si-16O2
184,cis-31P2-1H2
185,trans-31P2-1H-2H
186,trans-31P2-1H2


In [5]:
iso_formula

,0
0,(2H)2(16O)
1,(1H)2(16O)
2,(1H)2(18O)
3,(1H)2(17O)
4,(1H)(2H)(16O)
...,...
183,(28Si)(16O)2
184,cis-(31P)2(1H)2
185,trans-(31P)2(1H)(2H)
186,trans-(31P)2(1H)2


In [6]:
isotopologue

,0
0,Hewitt
1,POKAZATEL
2,HotWat78
3,HotWat78
4,VTT
...,...
183,OYT3
184,OY-Cis
185,OY-Trans
186,OY-Trans


In [7]:
molecule_single

,0
0,H2O
1,CO2
2,CO
3,CH4
4,NO
...,...
73,SiH2
74,SiO2
75,cis-P2H2
76,trans-P2H2


In [8]:
num_isotopologues

,0
0,5
1,13
2,6
3,1
4,6
...,...
73,1
74,1
75,1
76,2


In [9]:
molecule_repeated = pd.DataFrame()
molecule_num = len(molecule_single)

for j in tqdm_nb(range(molecule_num)):    
    molecule_repeated = molecule_repeated.append(pd.DataFrame((molecule_single.values[j] + ' ') * int(num_isotopologues.values[j])))
    
    
molecule_str = str(molecule_repeated.values).replace("[['"," ").replace("']\n ['"," ").replace("']]"," ").replace("+","_p")
molecule = pd.read_csv(StringIO(molecule_str), sep='\s+', header=None)
molecule



,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,H2O,H2O,H2O,H2O,H2O,CO2,CO2,CO2,CO2,CO2,...,AsH3,PF3,PH,CH3,SiH2,SiO2,cis-P2H2,trans-P2H2,trans-P2H2,H3O_p


In [10]:
molecule.values

array([['H2O', 'H2O', 'H2O', 'H2O', 'H2O', 'CO2', 'CO2', 'CO2', 'CO2',
        'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2',
        'CO', 'CO', 'CO', 'CO', 'CO', 'CO', 'CH4', 'NO', 'NO', 'NO',
        'NO', 'NO', 'NO', 'SO2', 'NH3', 'NH3', 'HNO3', 'OH', 'HF', 'HCl',
        'HCl', 'HCl', 'HCl', 'H2CO', 'N2', 'HCN', 'HCN', 'CH3Cl',
        'CH3Cl', 'H2O2', 'C2H2', 'PH3', 'H2S', 'C2H4', 'CS', 'CS', 'CS',
        'CS', 'CS', 'CS', 'CS', 'CS', 'H2', 'H2', 'LiH', 'LiH_p', 'FeH',
        'TiH', 'BeH', 'BeH', 'BeH', 'CaH', 'CrH', 'SiH', 'SiH', 'SiH',
        'SiH', 'AlH', 'AlH', 'AlH', 'NaH', 'NaH', 'MgH', 'MgH', 'MgH',
        'SiO', 'SiO', 'SiO', 'SiO', 'SiO', 'TiO', 'TiO', 'TiO', 'TiO',
        'TiO', 'MgO', 'MgO', 'MgO', 'MgO', 'MgO', 'H3_p', 'H3_p',
        'HeH_p', 'HeH_p', 'HeH_p', 'HeH_p', 'H2_p', 'C2', 'C2', 'C2',
        'CN', 'CH', 'CH', 'LiCl', 'LiCl', 'LiCl', 'LiCl', 'NaCl', 'NaCl',
        'NH', 'SO3', 'YO', 'VO', 'SH', 'SH', 'SH', 'SH', 'SH', 'AlO',
        'A

In [11]:
def_url = pd.DataFrame()

def_num = len(iso_slug)

for i in tqdm_nb(range(def_num)):
    def_url = def_url.append('http://www.exomol.com/db/' + molecule[i] + '/'
                             + iso_slug.values[i] + '/'+ isotopologue.values[i] + '/'
                             + iso_slug.values[i] + '__' + isotopologue.values[i] + '.def')

    
def_url['IsoFormula'] = iso_formula
def_url.columns = ['def url','IsoFormula']

def_url

,def url,IsoFormula
0,http://www.exomol.com/db/H2O/2H2-16O/Hewitt/2H...,(2H)2(16O)
1,http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL...,(1H)2(16O)
2,http://www.exomol.com/db/H2O/1H2-18O/HotWat78/...,(1H)2(18O)
3,http://www.exomol.com/db/H2O/1H2-17O/HotWat78/...,(1H)2(17O)
4,http://www.exomol.com/db/H2O/1H-2H-16O/VTT/1H-...,(1H)(2H)(16O)
...,...,...
183,http://www.exomol.com/db/SiO2/28Si-16O2/OYT3/2...,(28Si)(16O)2
184,http://www.exomol.com/db/cis-P2H2/cis-31P2-1H2...,cis-(31P)2(1H)2
185,http://www.exomol.com/db/trans-P2H2/trans-31P2...,trans-(31P)2(1H)(2H)
186,http://www.exomol.com/db/trans-P2H2/trans-31P2...,trans-(31P)2(1H)2


In [12]:
def_url.values[0:10]

array([['http://www.exomol.com/db/H2O/1H-2H-16O/VTT/1H-2H-16O__VTT.def',
        '(1H)(2H)(16O)'],
       ['http://www.exomol.com/db/CO2/12C-16O2/UCL-4000/12C-16O2__UCL-4000.def',
        '(12C)(16O)2'],
       ['http://www.exomol.com/db/CO2/13C-16O2/Zak/13C-16O2__Zak.def',
        '(13C)(16O)2'],
       ['http://www.exomol.com/db/CO2/16O-12C-18O/Zak/16O-12C-18O__Zak.def',
        '(16O)(12C)(18O)'],
       ['http://www.exomol.com/db/CO2/16O-12C-17O/Zak/16O-12C-17O__Zak.def',
        '(16O)(12C)(17O)'],
       ['http://www.exomol.com/db/CO2/16O-13C-18O/Zak/16O-13C-18O__Zak.def',
        '(16O)(13C)(18O)'],
       ['http://www.exomol.com/db/CO2/16O-13C-17O/Zak/16O-13C-17O__Zak.def',
        '(16O)(13C)(17O)'],
       ['http://www.exomol.com/db/CO2/12C-18O2/Zak/12C-18O2__Zak.def',
        '(12C)(18O)2'],
       ['http://www.exomol.com/db/CO2/17O-12C-18O/Zak/17O-12C-18O__Zak.def',
        '(17O)(12C)(18O)'],
       ['http://www.exomol.com/db/CO2/13C-18O2/Zak/13C-18O2__Zak.def',
        '(

In [13]:
def download_deffile(file_url):
    
    failed_list = [] 
    for _link in tqdm_nb(file_url['def url'].values):
        
        link = _link
        iso_slug = link.split('/')[-4]
        iso_formula = str(file_url[file_url['def url'].isin([link])]['IsoFormula'].values).replace("['","").replace("']","")
        inital_def_name = link.split('/')[-1]
        new_def_filename = iso_slug + '_' + iso_formula + '_' + inital_def_name
        print("Downloading file: %s" % new_def_filename)
        print(link)
 
        # Make folders for save doanloaded files.
        folder_name = './data/def/'
        if os.path.exists(folder_name):
            pass
        else:
            os.makedirs(folder_name, exist_ok=True)
        filename = os.path.join(folder_name, new_def_filename)
        
        try:
            r = requests.get(link, stream=True, verify=False)
        except Exception:
            failed_list.append(new_def_filename)
            print(' download failed. Go to download next one\n')
              
        # For compute the progess.
        total_size = int(r.headers['Content-Length'])
        temp_size = 0    
   
        # Download started.
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    temp_size += len(chunk)
                    f.write(chunk)
                    f.flush()
                    done = int(50 * temp_size / total_size)
                    sys.stdout.write("\r[%s%s] %d%%" % ('█' * done, ' ' * (50 - done), 100 * temp_size / total_size))
                    sys.stdout.flush()
        
        print(" Downloaded!\n")

    print("All def files downloaded!\n")    
    print("The files which are failed to download: \n")
    print(failed_list) # Record which file is failed to download.

    return


In [14]:
download_deffile(def_url)

http://www.exomol.com/db/H2O/2H2-16O/Hewitt/2H2-16O__Hewitt.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/H2O/1H2-18O/HotWat78/1H2-18O__HotWat78.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/H2O/1H2-17O/HotWat78/1H2-17O__HotWat78.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/H2O/1H-2H-16O/VTT/1H-2H-16O__VTT.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/CO2/12C-16O2/Zak/12C-16O2__Zak.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/CO2/13C-16O2/Zak/13C-16O2__Zak.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/CO2/16O-12C-18O/Zak/16O-12

[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/SiO/30Si-16O/EBJT/30Si-16O__EBJT.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/TiO/46Ti-16O/Toto/46Ti-16O__Toto.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/TiO/50Ti-16O/Toto/50Ti-16O__Toto.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/TiO/48Ti-16O/Toto/48Ti-16O__Toto.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/TiO/47Ti-16O/Toto/47Ti-16O__Toto.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/TiO/49Ti-16O/Toto/49Ti-16O__Toto.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/MgO/26Mg-16O/LiTY/26Mg-16O__LiTY.def
[██████████████████████████████████████████████████] 100% Downloaded!


http://www.exomol.com/db/MgF/24Mg-19F/MoLLIST/24Mg-19F__MoLLIST.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/MgF/25Mg-19F/MoLLIST/25Mg-19F__MoLLIST.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/MgF/26Mg-19F/MoLLIST/26Mg-19F__MoLLIST.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/OH_p/16O-1H_p/MoLLIST/16O-1H_p__MoLLIST.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/AsH3/75As-1H3/CYT18/75As-1H3__CYT18.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/PF3/31P-19F3/MCYTY/31P-19F3__MCYTY.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/PH/31P-1H/LaTY/31P-1H__LaTY.def
[██████████████████████████████████████████████████] 100% Downloaded!

http://www.exomol.com/db/CH3/12C-1H3/AYYJ/1

In [22]:
path = "./data/def"

def_col_name = ['c0', '#', 'c1', 'c2', 'c3', 'c4', 'c5']

for(dirpath,dirnames,files)in os.walk(path):
    tot = 0
    count = 0
    unc_def_filename = []
    for filename in files:
        filepath = os.path.join(dirpath, filename)
        tot += 1
        def_df =  pd.read_csv(filepath,sep='\s+', names=def_col_name, header=None)
        c1 = def_df['c1']
        if def_df[c1.isin(['Uncertainty'])]['c0'].values == '1':
            unc_def_filename.append(filename)
            count += 1            
        else:
            no_unc_def_filename = []
        
    print('There are ', tot, ' def files.\n')
    print('The uncertainty availability shows YES in the following ', count, ' def files:\n', unc_def_filename)
    print('\nThe uncertainty availability does not exit or shows NO in other ', tot - count, 'def files.')
                    

D:\Downloads\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  app.launch_new_instance()


There are  188  def files.

The uncertainty availability shows YES in the following  10  def files:
 ['AlH_(27Al)(1H)_27Al-1H__AlHambra.def', 'C2H2_(12C)2(1H)2_12C2-1H2__aCeTY.def', 'C2_(12C)2_12C2__8states.def', 'H2O_(1H)2(16O)_1H2-16O__POKAZATEL.def', 'H3O_p_(1H)3(16O)+_1H3-16O_p__eXeL.def', 'TiO_(46Ti)(16O)_46Ti-16O__Toto.def', 'TiO_(47Ti)(16O)_47Ti-16O__Toto.def', 'TiO_(48Ti)(16O)_48Ti-16O__Toto.def', 'TiO_(49Ti)(16O)_49Ti-16O__Toto.def', 'TiO_(50Ti)(16O)_50Ti-16O__Toto.def']

The uncertainty availability does not exit or shows NO in other  178 def files.


In [23]:
unc_def_filename

['AlH_(27Al)(1H)_27Al-1H__AlHambra.def',
 'C2H2_(12C)2(1H)2_12C2-1H2__aCeTY.def',
 'C2_(12C)2_12C2__8states.def',
 'H2O_(1H)2(16O)_1H2-16O__POKAZATEL.def',
 'H3O_p_(1H)3(16O)+_1H3-16O_p__eXeL.def',
 'TiO_(46Ti)(16O)_46Ti-16O__Toto.def',
 'TiO_(47Ti)(16O)_47Ti-16O__Toto.def',
 'TiO_(48Ti)(16O)_48Ti-16O__Toto.def',
 'TiO_(49Ti)(16O)_49Ti-16O__Toto.def',
 'TiO_(50Ti)(16O)_50Ti-16O__Toto.def']

## Get Download Links

In [24]:
molecule_str = []
iso_formula_str = []
isotopologue_str = []
iso = []
api_url = []
target_link = []

for i in range(len(unc_def_filename)):
    molecule_str.append(unc_def_filename[i].replace('_p','+').split('_')[0].replace('+','_p'))
    iso_formula_str.append(unc_def_filename[i].replace('_p','+').split('_')[1])
    isotopologue_str.append(unc_def_filename[i].split('_')[-1].split('.')[0])
    
    _iso = (iso_formula_str[i], isotopologue_str[i])
    iso.append(_iso)
    
    api_url.append('http://exomol.com/api/?molecule=*&datatype=linelist'.replace('*',molecule_str[i]))
    
print(iso)
api_url

[('(27Al)(1H)', 'AlHambra'), ('(12C)2(1H)2', 'aCeTY'), ('(12C)2', '8states'), ('(1H)2(16O)', 'POKAZATEL'), ('(1H)3(16O)+', 'eXeL'), ('(46Ti)(16O)', 'Toto'), ('(47Ti)(16O)', 'Toto'), ('(48Ti)(16O)', 'Toto'), ('(49Ti)(16O)', 'Toto'), ('(50Ti)(16O)', 'Toto')]


['http://exomol.com/api/?molecule=AlH&datatype=linelist',
 'http://exomol.com/api/?molecule=C2H2&datatype=linelist',
 'http://exomol.com/api/?molecule=C2&datatype=linelist',
 'http://exomol.com/api/?molecule=H2O&datatype=linelist',
 'http://exomol.com/api/?molecule=H3O_p&datatype=linelist',
 'http://exomol.com/api/?molecule=TiO&datatype=linelist',
 'http://exomol.com/api/?molecule=TiO&datatype=linelist',
 'http://exomol.com/api/?molecule=TiO&datatype=linelist',
 'http://exomol.com/api/?molecule=TiO&datatype=linelist',
 'http://exomol.com/api/?molecule=TiO&datatype=linelist']

Get the download links of states.bz2 files and trans.bz2 files from webset.

In [35]:
def get_target_url():
    
    """
    Get the download url from api.

    """
    
    file_url = []
    for i in range(len(iso)):
        
        response = requests.get(api_url[i])

        if(response.status_code != 200):
            print('ExoMol API Error' + str(response.status_code))

        # If the obtained status code is 200, it is correct.
        else:
            content = response.text            # Get the relevant content.
            json_dict = json.loads(content)    # Convert json into dictionary.

            # Extract files information from dictionary and convert them into list
            iso_slug = iso[i][0]
            isotopologue = iso[i][1]
            json_list = json_dict[iso_slug]['linelist'][isotopologue]['files']

            print('The number of downloading files for', iso_slug, isotopologue, ': ', len(json_list))
            print("Download links:")
            
            url_show = []
            for j in range(len(json_list)):
                link = json_list[j].get('url')
                try:
                    if((link.endswith('states.bz2') or link.endswith('trans.bz2'))):
                    #if((link.endswith('states.bz2'))):
                        file_url.append("http://www." + link)
                        url_show.append("http://www." + link)
                except KeyError:
                    print('Keyerror, keep going!')
                    
    
        for k in url_show:
            print(k)

    return file_url


In [36]:
target_link = get_target_url()

The number of downloading files for (27Al)(1H) AlHambra :  2
Download links:
http://www.exomol.com/db/AlH/27Al-1H/AlHambra/27Al-1H__AlHambra.trans.bz2
http://www.exomol.com/db/AlH/27Al-1H/AlHambra/27Al-1H__AlHambra.states.bz2
The number of downloading files for (12C)2(1H)2 aCeTY :  101
Download links:
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY.states.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00000-00100.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00100-00200.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00200-00300.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00300-00400.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00400-00500.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00500-00600.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY__00600-00700.trans.bz2
http://www.exomol.com/db/C2H2/12C2-1H

http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08100-08200.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08200-08300.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08300-08400.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08400-08500.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08500-08600.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08600-08700.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08700-08800.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08800-08900.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__08900-09000.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__09000-09100.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__09100-09200.trans.bz2
http://www.exomol.com

http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__26500-26600.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__26600-26700.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__26700-26800.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__26800-26900.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__26900-27000.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27000-27100.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27100-27200.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27200-27300.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27300-27400.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27400-27500.trans.bz2
http://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__27500-27600.trans.bz2
http://www.exomol.com

http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__03600-03700.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__03700-03800.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__03800-03900.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__03900-04000.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04000-04100.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04100-04200.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04200-04300.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04300-04400.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04400-04500.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04500-04600.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04600-04700.trans.bz2
http://www.exomol.com/db/H3O_p/1H3-16O_p/eXeL/1H3-16O_p__eXeL__04

## Download Files

We write all the download URLs into a text file, name it as $api__urls.txt$. 

In Linux, we use command 

```
wget -r -i /.../save_path/.../api__urls.txt
```

Download states.bz2 files and trans.bz2 files with download links. Save these files into correspoding folders.


In [40]:
url_path = './data/url'
url_filename = url_path + '/api__urls.txt'

if os.path.exists(url_path):
    pass
else:
    os.makedirs(url_path, exist_ok=True)

with open(url_filename, 'w') as file:
    file.write('\n'.join(target_link))